In [22]:
import os

# Путь к папке с данными
data_dir = 'C:\\Users\\gaziz\\Downloads\\archive.csv\\Agricultural-crops'

# Получите список подпапок (каждая подпапка соответствует классу)
class_folders = [f for f in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, f))]

# Создайте список для хранения изображений и соответствующих классов
images = []
labels = []

# Загрузите изображения из каждой подпапки
for class_folder in class_folders:
    class_path = os.path.join(data_dir, class_folder)
    for image_filename in os.listdir(class_path):
        image_path = os.path.join(class_path, image_filename)
        # Здесь можно выполнить дополнительные операции, такие как изменение размеров изображений и нормализацию
        images.append(image_path)
        labels.append(class_folder)

In [48]:
import pandas as pd
df = pd.DataFrame({"Image": images, "Label": labels})
df

,Image,Label
0,C:\Users\gaziz\Downloads\archive.csv\Agricultu...,almond
1,C:\Users\gaziz\Downloads\archive.csv\Agricultu...,almond
2,C:\Users\gaziz\Downloads\archive.csv\Agricultu...,almond
3,C:\Users\gaziz\Downloads\archive.csv\Agricultu...,almond
4,C:\Users\gaziz\Downloads\archive.csv\Agricultu...,almond
...,...,...
824,C:\Users\gaziz\Downloads\archive.csv\Agricultu...,wheat
825,C:\Users\gaziz\Downloads\archive.csv\Agricultu...,wheat
826,C:\Users\gaziz\Downloads\archive.csv\Agricultu...,wheat
827,C:\Users\gaziz\Downloads\archive.csv\Agricultu...,wheat


In [54]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
import numpy as np

X = df['Image']
y = df['Label']

# Split the data into train, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.1, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [100]:
from PIL import Image
import numpy as np

# Define a function to load, resize, and preprocess an image
def preprocess_image(file_path, target_size=(224, 224)):
    try:
        # Open the image
        img = Image.open(file_path)
        
        # Resize the image to the target size
        img = img.resize(target_size)
        
        # Convert the image to a NumPy array and normalize pixel values
        img_array = np.array(img)
        img_array = img_array / 255.0  # Normalize pixel values to the range [0, 1]

        return img_array
    except Exception as e:
        print(f"Error processing image {file_path}: {str(e)}")
        return None

# Preprocess train, validation, and test sets
X_train_processed = [preprocess_image(file_path) for file_path in X_train]
X_val_processed = [preprocess_image(file_path) for file_path in X_val]
X_test_processed = [preprocess_image(file_path) for file_path in X_test]

In [115]:
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.svm import SVC
# Initialize standard scaler for feature scaling
scaler = StandardScaler()

# Initialize Softmax Regression (Logistic Regression) with L2 regularization
softmax_reg = LogisticRegression(penalty='l2', multi_class='multinomial', solver='lbfgs', max_iter=1000, random_state=42)

# Initialize SVM with regularization (C parameter controls regularization strength)
svm_classifier = SVC(kernel='linear', C=1.0, random_state=42, max_iter=1000)

# Create a 5-fold cross-validation splitter
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Evaluate Softmax Regression with cross-validation
softmax_scores = cross_val_score(softmax_reg, X_train_processed, y_train, cv=kfold, scoring='accuracy')
print("Mean Accuracy for Softmax Regression:", np.mean(softmax_scores))

# Fit and evaluate SVM with cross-validation
svm_scores = cross_val_score(svm_classifier, X_train_processed, y_train, cv=kfold, scoring='accuracy')
print("Mean Accuracy for SVM:", np.mean(svm_scores))

Mean Accuracy for Softmax Regression: 0.061664429530201334
Mean Accuracy for SVM: 0.04558389261744967
